# Setup Code

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# imports
import torch
import torch.nn.functional as F
import numpy as np
import sys
import os
from matplotlib import pyplot as plt

In [ ]:
# select devices
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Bad to go!")
    DEVICE = torch.device("cpu")

Load config from config files.

In [ ]:
# load config
sys.path.append(os.getcwd())
# choose between ship, lego
import configs.ship, configs.lego
# change config file here
config = configs.lego

Load the dataset and show the first one.

In [ ]:
from nerf.data import load_blender
imgs, poses, int_mat = load_blender(config.datadir, device=DEVICE, scale_factor=2)
img_n, img_h, img_w = imgs.shape[:3]
# visualize
plt.imshow(np.array(imgs[0].to(device="cpu")))
plt.axis("off")
plt.title("demo image")
plt.show()
print("and its pose: ")
print(np.array(poses[0].to(device="cpu")))

Compute rays

In [ ]:
from nerf.graphics import compute_rays

rays_o, rays_d = compute_rays((img_h, img_w), int_mat, poses[0])
print("origin: ", rays_o[0,0])
print("normalized origin: ", F.normalize(rays_o[0,0], dim=0))
print("center of ray: ", rays_d[img_h//2,img_w//2])

In [ ]:
from nerf.graphics import queries_from_rays
samples = None
samples, depths = queries_from_rays(rays_o, rays_d, 2, 6, 8)
print(depths)

In [ ]:
# test posencode

from nerf.nerf_helper import PosEncode

L = 2
x = torch.tensor([[1.1, 1.2, 1.3], [2.1, 2.2, 2.3]])
enc_x = PosEncode(x, L)

print(enc_x)


# Train

In [ ]:
# One iteration of TinyNeRF (forward pass).